In [1]:
from src.problems.minimum_vertex_cover_vertices import MinimumVertexCoverVertices
from src.choices.logChoice import LogChoice
from src.choices.bigChoice import BigChoice
from src.choicesParameters.fixedParameters import FixedParameters
from src.choicesParameters.metaheuristicParameters import PSOParameters
from src.choicesParameters.transformerParameters import TransformerParametersSVD
from src.problems.utils import generate_random_graph, greedy_dominating_set
import random

In [2]:
choice = LogChoice()
choiceParameters = FixedParameters()

def fitness(p, problem : MinimumVertexCoverVertices, graph):
    choiceParameters.set(p[0])
    worst_case = None

    for _ in range(10):
        v = problem.run(graph)

        if worst_case is None or worst_case < v:
            worst_case = v
    
    return worst_case

In [3]:
X = []
Y = []

In [ ]:
problem = MinimumVertexCoverVertices(choice, choiceParameters)

for _ in range(100):
    graph = generate_random_graph(random.randint(3, 50))

    psoParameters = PSOParameters()
    score, p = psoParameters.fit(lambda p : fitness(p, problem, graph), choice.apply_constraints, 2, 10, 15)
    print(score, p)
    
    X.extend([[problem.embeddings[0]]])
    Y.extend([p])

In [6]:
transformerParameters = TransformerParametersSVD(1)
transformerParameters.remember_previous_internal_state = False

In [ ]:
preds, error = transformerParameters.train(X, Y)
print(error)

In [8]:
def test(choice, choiceParameters):
    problem = MinimumVertexCoverVertices(choice, choiceParameters)
    problemGreedy = MinimumVertexCoverVertices(BigChoice(), FixedParameters())

    losses = 0
    ties = 0
    wins = 0

    for _ in range(100):
        graph = generate_random_graph(random.randint(3, 20))
        v = problem.run(graph)
        g = problemGreedy.run(graph)

        if v == g:
            ties += 1
        elif v < g:
            wins += 1
        else:
            losses += 1

    return losses, ties, wins

In [9]:
print(test(choice, transformerParameters))

(10, 90, 0)
